In [1]:
import cv2
from ultralytics import YOLO
import polars as pl
from tqdm import tqdm
import os

In [2]:
model = YOLO("../weights/yolov10s.pt")  # Замените на yolov8m.pt, yolov8l.pt и т.д., если необходимо

# Пример использования функции
input_video_path = "../test_videos/test.mp4"
output_video_path = "../output_videos/video_cluttering_output.mp4"
annotations_csv_path = "../annotations/cluttering_annotations.csv" 

objects_to_detect = ["car", "person"]  # Замените на необходимые объекты

In [4]:
def process_video(input_path, output_path, objects_to_detect, annotations_csv):

    # Захват видео
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Не удалось открыть видео: {input_path}")
        return

    # Получение параметров видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Инициализация видеописьма
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Можно изменить на нужный кодек
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Список для хранения аннотаций
    annotations = []

    # Обработка кадров с прогресс-баром
    for frame_idx in tqdm(range(total_frames), desc="Обработка видео"):
        ret, frame = cap.read()
        if not ret:
            break

        # Детекция объектов
        results = model.predict(frame, verbose=False)

        # Перебор обнаруженных объектов
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls[0].item())
                label = model.names[cls_id]
                confidence = box.conf[0].item()
                bbox = box.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2]

                if label in objects_to_detect:
                    # Рисуем красную рамку
                    x1, y1, x2, y2 = map(int, bbox)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                    # Добавляем аннотацию
                    annotations.append({
                        "frame_number": frame_idx,
                        "object": label,
                        "x1": x1,
                        "y1": y1,
                        "x2": x2,
                        "y2": y2,
                        "confidence": confidence
                    })

        # Запись обработанного кадра
        out.write(frame)
        cv2.imshow("De-cluttering", frame)

        # Завершение по нажатию клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Закрытие видео
    cap.release()
    out.release()

    # Создание и сохранение CSV с помощью Polars
    if annotations:
        df = pl.DataFrame(annotations)
        df.write_csv(annotations_csv)
        print(f"Аннотации сохранены в {annotations_csv}")
    else:
        print("Обнаруженные объекты не найдены. CSV-файл не создан.")

    print(f"Обработанное видео сохранено в {output_path}")

# Вызов функции обработки видео
process_video(input_video_path, output_video_path, objects_to_detect, annotations_csv_path)

Обработка видео: 100%|██████████| 911/911 [03:21<00:00,  4.53it/s]

Аннотации сохранены в ../annotations/cluttering_annotations.csv
Обработанное видео сохранено в ../output_videos/video_cluttering_output.mp4
